In [ ]:
Ai = [8, 11, 14, 15, 16, 17, 18, 19, 20, 21, 26, 27, 28, 40, 45, 46, 55, 56, 61]
Ar = [2, 4, 5, 24, 25, 30, 40, 44, 45]
Sy = [2, 4, 5, 6, 7, 13, 20, 23, 24, 25, 29, 30, 33, 38, 40, 41, 42, 43, 44, 45]
Ap = [1, 3, 6, 7, 13, 17, 20, 21, 22, 27, 32, 33, 35, 38, 39, 40, 41, 42, 43]
Th = [3, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 27, 28, 31, 37, 40, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 70, 71, 72, 73]
Ec = [10, 11, 17, 27, 40, 46, 48, 49, 60, 71, 72]
Ro = [4, 20, 21, 27, 40, 44, 45, 46, 73]
Da = [9, 11, 14, 17, 18, 19, 27, 33, 34, 40, 46, 48, 49, 55, 56, 60, 61, 71, 72, 73]
Bi = [34, 36, 40]
Is = [9, 12, 32, 40, 41]

In [14]:
using CSV

raw = CSV.read("courses_and_GPA.csv")
(m,n) = size(raw)

courses = raw[:,1]
gpa = convert(Matrix{Float64}, raw[:,2:n])
classnum = length(gpa)
pre = [[0],[0],[0],[2],[2,3],[3],[3],[69,70,3],[51],[51],[49],[51],[6,7],[55,3],[51,59,3],[70,6],
    [3,51],[51],[18,3],[69,1],[45,51],[3,69],[5,6],[5,6],[5,6],[1],[3,68],[23],[47,3,5],[4,5],
    [6,47],[69,6],[5,6],[55,73],[1],[3,69],[70,6],[0],[3,51],[0],[24],[24],[32],[0],[44],[44],
    [69],[47],[71,73],[69],[69],[58],[52],[47],[69],[47],[0],[57],[47],[47],[51],[47],[53],[56,57],
    [64],[64],[56],[0],[68],[0],[47],[48],[68]]                                 

# create P (pre-requisite matrix as in Write Up)
# so that can use P*x to simplify stuff
P = zeros(Int8, classnum, classnum)
for i in 1:size(pre,1)
    # diagnal = 1 if i has no pre-req
    if pre[i][1] == 0
        P[i,i] = 1  
    else  # else loop through pre-req of i see if it has a pre-req
        for k in 1:size(pre[i],1) 
            P[i,pre[i][k]] = 1 # assign 1 if k is a pre-req 
        end
    end
end
           
P # check P 

73×73 Array{Int8,2}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  0  0  0  0  0  0     

In [15]:
# conversion to binary arrays as defined 

basic_cs0 = [2,3,5,6,70]
basic_math0 = [68,69]
add_math0 = [55,73,8,10,14,15,18,19,46,47,50,51,52,53,54,56,58,60,61,62,12,63,64,65,66,67,48,49,71,72]
theory0 = [37,16]
soft_hard0 = [7,13,23,24,30,33,41,42]
app0 = [8,9,14,15,17,18,22,27,28,29,32,35,]
elect0 = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,40]
len = length(gpa);


# function converts above kind of array into ones like in the final project write up math model
# so that multiplication are easier 
function array_to_bin(arraytoconvert)
    converted=zeros(classnum)
    for i in 1:classnum
        for j in 1:size(arraytoconvert,1)
            if i == arraytoconvert[j]
                converted[i] = 1
            end
        end
    end
    return converted
end


# converts all above data
basic_cs = array_to_bin(basic_cs0)
basic_math = array_to_bin(basic_math0)
add_math =array_to_bin(add_math0)
theory = array_to_bin(theory0)
soft_hard = array_to_bin(soft_hard0)
app = array_to_bin(app0)
elect = array_to_bin(elect0)


73-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# you can change these below integer params and hit rum.

In [84]:
# you can change these below integer params and hit rum.
# These params are independent of the objective (max GPA, and min Classes) so you can change these
NUM_SEMESTER = 8  # int (max semesters to complete program)
SEM_COURSES_UP = 9  # semester courses upper bound
SEM_COURSES_LO = 0  # semester courses lower bound

0

In [85]:

# Every requirement list must x8 (or the semester number)
# instantiate for concact
basic_cs1 = Float64[]
basic_math1 = Float64[]
add_math1 = Float64[]
theory1 = Float64[]
soft_hard1 = Float64[]
app1 = Float64[]
elect1 = Float64[]
gpa1 = Float64[] # need to edit GPA as it will be easier to multiply with x

# # concact vectors 
for i in 1:NUM_SEMESTER
    basic_cs1 = vcat(basic_cs1, basic_cs)
    basic_math1 = vcat(basic_math1, basic_math)
    add_math1 = vcat(add_math1, add_math)
    theory1 = vcat(theory1, theory)
    soft_hard1 = vcat(soft_hard1, soft_hard)
    app1 = vcat(app1, app)
    elect1 = vcat(elect1, elect)
    gpa1 = vcat(gpa1, gpa)
end

In [86]:
# using Gurobi here
using JuMP, Cbc, Gurobi, LinearAlgebra

# m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0)) Cdc didn't work with MathOptInterface.ScalarQuadraticFunction

m = Model(with_optimizer(Gurobi.Optimizer, NumericFocus=3,BarHomogeneous=1))

# Be careful when changing below, it might run into unbounded 
TOT_COURSES_UP = 15 # total courses upper bound()
TOT_COURSES_LO = 15  # total courses lower bound

@variable(m, x[1:len*NUM_SEMESTER], Bin)  # create a 73 * 8 (semester number) course list
@variable(m, satisfied[1:NUM_SEMESTER, 1:len])
@variable(m, upper_bound)

# basic catogorical course requirement
@constraint(m, basic_cs1'*x >= 5)  # why were they == 5 before? 
@constraint(m, basic_math1'*x >= 2) # why were they == 2 before? 
@constraint(m, add_math1'*x >= 2)
@constraint(m, theory1'*x >= 1)
@constraint(m, soft_hard1'*x >= 2)
@constraint(m, app1'*x >= 1)
@constraint(m, elect1'*x >= 2)


# 1st semester course load preference
@constraint(m, sum(x[1:73]) <= SEM_COURSES_UP) # upper bound
@constraint(m, sum(x[1:73]) >= SEM_COURSES_LO) # lower bound

taken = x[1:73]  # keeping track of a list of taken courses to avoid repeats, time(semester) invariant

# # # keeping track of a list of satisfied courses for pre-req requirement
# # # each semester deffers, thus must have a matrix
@constraint(m, satisfied[1,1:len] .== diag(P))
for i in 1:(NUM_SEMESTER-1)
    # A class cannot count for more than one degree requirement (no repeats)
    @constraint(m, taken' * x[(73*(i)+1):73*(i+1)] == 0)
    taken = taken + x[(73*(i)+1):73*(i+1)] # update list
    # Must satisfy prerequisite requirements
    @constraint(m, satisfied[i,1:len] - x[(73*(i-1)+1):73*i] .>= 0)
    @constraint(m, satisfied[i+1,1:len] .== satisfied[i,1:len] + P*x[(73*(i-1)+1):73*i]) # update list
    # semester course load preference (except for 1st semester)
    @constraint(m, sum(x[(73*(i)+1):73*(i+1)]) <= SEM_COURSES_UP) # upper bound
    @constraint(m, sum(x[(73*(i)+1):73*(i+1)]) >= SEM_COURSES_LO) # lower bound
end

# courses total preference 
@constraint(m, sum(x[i] for i in 1:len*NUM_SEMESTER) <= TOT_COURSES_UP) # upper bound
@constraint(m, sum(x[i] for i in 1:len*NUM_SEMESTER) >= TOT_COURSES_LO) # lower bound
@constraint(m, sum(x[i] for i in 1:len*NUM_SEMESTER) <= upper_bound)

# starting course preference
@constraint(m, x[2] == 1)  # must take CS 252 first semester
@constraint(m, x[3] == 1)  # must take CS 300 first semester
# class at x[142] is math 222 in second semester
# so I am setting constraint that math 222 must
# be taken in the second semester
# note you cant set math 222 to first semester since it has math 221
# as a pre-req, and you will get an unbounded solution
@constraint(m, x[142] == 1) # 142 = 69+tot_classes(73)

Academic license - for non-commercial use only


x[142] == 1.0

# Maximize GPA

In [87]:
# Maximize GPA
# run this cell to maximize the GPA and plot results
@objective(m, Max, (gpa1'*x)[1]);
optimize!(m)
# printing out final schedule.
opt_gpa = 0
for i in 1:len*NUM_SEMESTER
    n = i%len
    if n == 0  
        n = len # because Julia starts at 1
    elseif n == 1 # because Julia starts at 1
        println("=======================")
        print("Starting Semester ")
        println(trunc(Int,(i-1)/len+1))
        println("=======================")
    end
    if (value(x[i]) > 0)
        print(" - ")
        print(courses[n])
        print(" - ")
        println(round(gpa[n],digits=1))
        opt_gpa += gpa[n]
    end
end
println("")

# printing out schedule
print("Overall Predicted GPA:")
println((gpa1'*JuMP.value.(x))/sum(JuMP.value.(x)))
println("")


# printing out courses graduation satifaction
println("Below are how the classes fit the graduation requirement:")
print("- Basic CS classes: ")
basic_cs_list = basic_cs1+value.(x)
# loop through basic_cs_list
for i in 1:length(basic_cs_list)
    if basic_cs_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Basic Math classes: ")
basic_math_list = basic_math1+value.(x)
# loop through basic_cs_list
for i in 1:length(basic_math_list)
    if basic_math_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Additional Math classes: ")
add_math_list = add_math1+value.(x)
# loop through basic_cs_list
for i in 1:length(add_math_list)
    if add_math_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Computer theory classes: ")
theory_list = theory1+value.(x)
# loop through basic_cs_list
for i in 1:length(theory_list)
    if theory_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Software/hardware classes: ")
soft_hard_list = soft_hard1+value.(x)
# loop through basic_cs_list
for i in 1:length(soft_hard_list)
    if soft_hard_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Application classes: ")
app_list = app1+value.(x)
# loop through basic_cs_list
for i in 1:length(app_list)
    if app_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Elective classes: ")
elect_list = elect1+value.(x)
# loop through basic_cs_list
for i in 1:length(elect_list)
    if elect_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 1124 rows, 1169 columns and 6523 nonzeros
Model fingerprint: 0xb6c50d21
Model has 7 quadratic constraints
Variable types: 585 continuous, 584 integer (584 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 819 rows and 736 columns
Presolve time: 0.02s
Presolved: 305 rows, 433 columns, 2502 nonzeros
Variable types: 0 continuous, 433 integer (416 binary)

Root relaxation: objective 4.977654e+01, 55 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      49.7765410   49.77654  0.00%     -    0s

Explored 0 nodes (55 simplex iterations) in 0.03 second

In [92]:
value.(satisfied[8,70])

1.0

# Minimize courses 

In [81]:
# Minimize courses 
# run this cell to minimize courses and plot results
@objective(m, Min, upper_bound);
optimize!(m)

# printing out final results. all the strange +1 -1 is because Julia starts at 1
opt_gpa = 0
for i in 1:len*NUM_SEMESTER
    n = i%len
    if n == 0  
        n = len # because Julia starts at 1
    elseif n == 1 # because Julia starts at 1
        println("=======================")
        print("Starting Semester ")
        println(trunc(Int,(i-1)/len+1))
        println("=======================")
    end
    if (value(x[i]) > 0)
        print(" - ")
        print(courses[n])
        print(" - ")
        println(round(gpa[n],digits=1))
        opt_gpa += gpa[n]
    end
end
println("")

# printing out schedule
print("Overall Predicted GPA:")
println((gpa1'*JuMP.value.(x))/sum(JuMP.value.(x)))
println("")


# printing out courses graduation satifaction
println("Below are how the classes fit the graduation requirement:")
print("- Basic CS classes: ")
basic_cs_list = basic_cs1+value.(x)
# loop through basic_cs_list
for i in 1:length(basic_cs_list)
    if basic_cs_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Basic Math classes: ")
basic_math_list = basic_math1+value.(x)
# loop through basic_cs_list
for i in 1:length(basic_math_list)
    if basic_math_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Additional Math classes: ")
add_math_list = add_math1+value.(x)
# loop through basic_cs_list
for i in 1:length(add_math_list)
    if add_math_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Computer theory classes: ")
theory_list = theory1+value.(x)
# loop through basic_cs_list
for i in 1:length(theory_list)
    if theory_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Software/hardware classes: ")
soft_hard_list = soft_hard1+value.(x)
# loop through basic_cs_list
for i in 1:length(soft_hard_list)
    if soft_hard_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Application classes: ")
app_list = app1+value.(x)
# loop through basic_cs_list
for i in 1:length(app_list)
    if app_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end
println("")
print("- Elective classes: ")
elect_list = elect1+value.(x)
# loop through basic_cs_list
for i in 1:length(elect_list)
    if elect_list[i] == 2
        n = i % len  # convert to course numbers
        print(courses[n])
        print("  ")
    end
end

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 1124 rows, 1169 columns and 6523 nonzeros
Model fingerprint: 0xfe288caa
Model has 7 quadratic constraints
Variable types: 585 continuous, 584 integer (584 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]

MIP start from previous solve produced solution with objective 30 (0.00s)
Loaded MIP start from previous solve with objective 30

Presolve removed 809 rows and 721 columns
Presolve time: 0.03s
Presolved: 315 rows, 448 columns, 3387 nonzeros
Variable types: 0 continuous, 448 integer (431 binary)

Root relaxation: objective 1.500000e+01, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      15.0000000   